In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader
from helpers import read_hdf5, CombineDataset
from lenshelpers import Residual

from model.helpers_model import NeuralNet

import warnings
warnings.filterwarnings('ignore') 

from sklearn.metrics import precision_recall_curve, f1_score, auc, accuracy_score, roc_curve, roc_auc_score

In [2]:
# Build the four classes 
config_repo_model = 'data/configFile/config_model'
ratio = 0.75
percent = 0.01
size = 3000

res = Residual()
res.build(size, ratio = ratio, per_error = percent)

print('Data Generation Finished')

Data Generation Finished


In [3]:
str_ID = "P"+str(int(percent*100))+"R"+str(int(ratio*100))
[final_array, metadata] = read_hdf5(str_ID, path = "data/dataSet/")
metadata ['ID'] = np.arange(0,final_array.shape[0])

data_set = CombineDataset(metadata,'ID','class',final_array)

print('Reading Data Finished')
data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)

batch_size = 50
max_epoch = 70


loader_train = DataLoader(data_train, batch_size = batch_size, shuffle = True, 
                          num_workers = 0, drop_last=True)

loader_test = DataLoader(data_test, batch_size = batch_size, shuffle = True, 
                         num_workers = 0, drop_last=True)

Reading Data Finished


In [ ]:
metric = 'auc'
netacc = NeuralNet('BasicCNN', 'Adam')
for epoch in range(max_epoch):
    netacc.train(loader_train)
    res = netacc.test(loader_test, metric = metric)
    
print('Finished Training')
epoch = netacc.load_checkpoint()

with torch.no_grad():
    predictions = []
    targets = []
    for data in loader_test:
        images, meta, labels = data
        outputs = netacc.net(images)

        predictions.extend(outputs.cpu().numpy())
        targets.extend(labels.cpu().numpy())
        result = netacc.calculate_metrics(np.round(np.array(predictions)), np.array(targets))

epoch: 1.000, auc: 0.540
epoch: 2.000, auc: 0.554
epoch: 3.000, auc: 0.574
epoch: 4.000, auc: 0.594
epoch: 5.000, auc: 0.616
epoch: 6.000, auc: 0.653
epoch: 7.000, auc: 0.693
epoch: 8.000, auc: 0.695
epoch: 9.000, auc: 0.707
epoch: 10.000, auc: 0.712
epoch: 12.000, auc: 0.723
epoch: 14.000, auc: 0.749
epoch: 15.000, auc: 0.762
epoch: 16.000, auc: 0.763
epoch: 17.000, auc: 0.782
epoch: 18.000, auc: 0.788
epoch: 19.000, auc: 0.803
epoch: 20.000, auc: 0.818
epoch: 21.000, auc: 0.821
epoch: 22.000, auc: 0.829
epoch: 24.000, auc: 0.835
epoch: 25.000, auc: 0.840
epoch: 30.000, auc: 0.849
epoch: 36.000, auc: 0.859


In [ ]:
# calculate precision-recall curve
precision0, recall0, thresholds0 = precision_recall_curve(np.array(targets)[:,0], np.array(predictions)[:,0])
precision1, recall1, thresholds1 = precision_recall_curve(np.array(targets)[:,1], np.array(predictions)[:,1])

plt.plot(recall0, precision0, label='Mass error')
plt.plot(recall1, precision1, label='Source error')
# axis labels
plt.xlabel('Recall[-]')
plt.ylabel('Precision[-]')
# show the legend
plt.legend()
# show the plot

plt.savefig('figures/metric/'+metric+'RecallPrecision.jpeg')
plt.grid()
plt.show()

In [ ]:
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(np.array(targets)[:,0], np.array(predictions)[:,0])
lr_fpr, lr_tpr, _ = roc_curve(np.array(targets)[:,1], np.array(predictions)[:,1])
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, label='Mass error')
plt.plot(lr_fpr, lr_tpr, label='Source error')
# axis labels
plt.xlabel('False Positive Rate[-]')
plt.ylabel('True Positive Rate[-]')
# show the legend
plt.legend()
plt.grid()
# show the plot
plt.savefig('figures/metric/'+metric+'ROC.jpeg')
plt.show()

txt = "AUC score: {:.3f}, with "+metric+" metric" 
print(txt.format(result['auc']))